In [ ]:
import numpy as np
import copy
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import time
import multiprocessing as mp
import joblib
# from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras import optimizers
# from keras.constraints import maxnorm
from numpy.random import seed
import pandas as pd
#from tensorflow import set_random_seed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/TA

Mounted at /content/drive
/content/drive/MyDrive/TA


In [ ]:
df =  pd.read_csv('Train.csv')
df

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,Pic50
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.000000
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.481486
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,9.455932
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,9.154902
4,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.396374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.823909
192,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3.602060
193,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3.602060
194,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.397940


In [ ]:
x = df
y = df["Pic50"]

In [ ]:
x

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,Pic50
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.000000
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.481486
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,9.455932
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,9.154902
4,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.396374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.823909
192,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3.602060
193,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3.602060
194,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.397940


In [ ]:
y

,Pic50
0,7.000000
1,7.481486
2,9.455932
3,9.154902
4,6.396374
...,...
191,8.823909
192,3.602060
193,3.602060
194,8.397940


In [ ]:
data_test = pd.read_csv("Test.csv")
data_test

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,Pic50
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.000000
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,4.552842
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.481486
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,9.376751
4,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.251812
5,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,4.958607
6,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.769551
7,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.000000
8,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.939302
9,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.346787


In [ ]:
x_test = data_test.drop("Pic50", axis = 1)

y_test = data_test.iloc[:,[-1]]

In [ ]:
x_test

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y_test

,Pic50
0,7.000000
1,4.552842
2,8.481486
3,9.376751
4,8.251812
5,4.958607
6,8.769551
7,8.000000
8,6.939302
9,7.346787


In [ ]:
print(df.info())  # Untuk dataset training
print(data_test.info())  # Untuk dataset testing

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Columns: 882 entries, PubchemFP0 to Pic50
dtypes: float64(1), int64(881)
memory usage: 1.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Columns: 882 entries, PubchemFP0 to Pic50
dtypes: float64(1), int64(881)
memory usage: 344.7 KB
None


In [ ]:
print(x.head())
print(y.head())
print(x_test.head())
print(y_test.head())

   PubchemFP0  PubchemFP1  PubchemFP2  PubchemFP3  PubchemFP4  PubchemFP5  \
0           1           1           1           0           0           0   
1           1           1           1           0           0           0   
2           1           1           1           0           0           0   
3           1           1           1           1           0           0   
4           1           1           1           0           0           0   

   PubchemFP6  PubchemFP7  PubchemFP8  PubchemFP9  ...  PubchemFP872  \
0           0           0           0           1  ...             0   
1           0           0           0           1  ...             0   
2           0           0           0           1  ...             0   
3           0           0           0           1  ...             0   
4           0           0           0           1  ...             0   

   PubchemFP873  PubchemFP874  PubchemFP875  PubchemFP876  PubchemFP877  \
0             0             0

In [ ]:
feature_poly = joblib.load("selected_feature_poly.pkl")

In [ ]:
missing_features = [feat for feat in feature_poly if feat not in x.columns]
if missing_features:
    print("Missing features:", missing_features)
else:
  print("Raihan ganteng")

Raihan ganteng


In [ ]:
X_train_poly = x[feature_poly]

In [ ]:
X_test_poly = x_test[feature_poly]

In [ ]:
best_estimator_poly = joblib.load("grid_poly.best_estimator.pkl")

In [ ]:
print(X_test_poly.isnull().sum())

PubchemFP1      0
PubchemFP2      0
PubchemFP3      0
PubchemFP4      0
PubchemFP5      0
               ..
PubchemFP871    0
PubchemFP872    0
PubchemFP874    0
PubchemFP876    0
PubchemFP879    0
Length: 407, dtype: int64


In [ ]:
def r2_score(y, y_pred, y_train=None):
    sse = 0; sst = 0
    if y_train is None:
        y_mean = np.mean(y)
    else:
        y_mean = np.mean(y_train)
    for i in range(len(y)):
        sse += (y[i] - y_pred[i]) ** 2
        sst += (y[i] - y_mean) ** 2
    r2_score = 1 - (sse / sst)
    return r2_score

In [ ]:
def q2_loo(model, X, y):
    loo = LeaveOneOut()
    y_pred = []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        tmp = model.predict(X_test)
        #tmp = scaler_y.inverse_transform(tmp)
        y_pred.append(list(tmp)[0])
    r2 = r2_score(y_train, y_pred)
    return r2

In [ ]:
def mse_score(y, y_pred):
    if isinstance(y, np.ndarray) or isinstance(y, list):
        n_data = len(y)
        sum_err = 0
        for i in range(n_data):
            err = y[i] - y_pred[i]
            sum_err += (err ** 2)
        return sum_err / n_data
    else:
        return (y - y_pred) ** 2

In [ ]:
def normalize(X):
    max_ = np.max(X, axis=0)
    min_ = np.min(X, axis=0)
    X_norm = (X - min_) / (max_ - min_)
    return max_, min_, X_norm

In [ ]:
def standardize(X):
    mean_ = np.mean(X, axis=0)
    std_ = np.std(X, axis=0)
    X_norm = (X - mean_) / std_
    return X_norm, mean_, std_

In [ ]:
def qsar_param(y, y_pred, d_r2m=True):
    results = []
    _, _, y = normalize(y)
    _, _, y_pred = normalize(y_pred)
    y_mean = np.mean(y); y_pred_mean = np.mean(y_pred)
    # calculate r2
    num = 0; den_1 = 0; den_2 = 0
    for i in range(len(y)):
        num += (y[i] - y_mean) * (y_pred[i] - y_pred_mean)
        den_1 += (y_pred[i] - y_pred_mean) ** 2
        den_2 += (y[i] - y_mean) ** 2
    r2 = num ** 2 / (den_1 * den_2)
    results = {"r2": r2}
    # calculate k and k_dash
    n_data = len(y)
    dot_ = 0; y_pred2 = 0; y2 = 0
    for i in range(n_data):
        dot_ += (y[i] * y_pred[i])
        y_pred2 += y_pred[i] ** 2
        y2 += y[i] ** 2
    k = np.sum(dot_) / np.sum(y_pred2)
    k_dash = np.sum(dot_) / np.sum(y2)
    results["k"] = k
    results["k_dash"] = k_dash
    # calculate r2_0 and r2_0_dash
    num = 0; num_dash = 0; den = 0; den_dash = 0
    for i in range(n_data):
        num += (y[i] - (k * y_pred[i])) ** 2
        num_dash += (y_pred[i] - (k_dash * y[i])) ** 2
        den += (y[i] - y_mean) ** 2
        den_dash += (y_pred[i] - y_pred_mean) ** 2
    r2_0 = 1 - (num / den)
    r2_0_dash = 1 - (num_dash / den_dash)
    #results.append(r2_0)
    #results.append(r2_0_dash)
    r2r0 = (r2 - r2_0)/r2
    r2r0_dash = (r2 - r2_0_dash)/r2
    results["r2r0"] = r2r0
    results["r2r0_dash"] = r2r0_dash
    r0r0_dash = np.abs(r2_0 - r2_0_dash)
    results["r0r0_dash"] = r0r0_dash
    # calculate rm2 and rm2_dash
    rm2 = r2 * (1 - np.sqrt(r2 - r2_0))
    rm2_dash = r2 * (1 - np.sqrt(r2 - r2_0_dash))
    results["rm2"] = rm2
    results["rm2_dash"] = rm2_dash
    #results.append(rm2)
    #results.append(rm2_dash)
    # calculate rm2_bar and d_rm2
    rm2_bar = (rm2 + rm2_dash) / 2
    d_rm2 = np.abs(rm2 - rm2_dash)
    results["rm2_bar"] = rm2_bar
    results["d_rm2"] = d_rm2
    return results

In [ ]:
# Prediksi menggunakan model terbaik (poly kernel) untuk data pelatihan
y_pred_train_poly = best_estimator_poly.predict(X_train_poly.values)
y_pred_test_poly = best_estimator_poly.predict(X_test_poly.values)

# # Evaluasi R² untuk data pelatihan
# r2_train = r2_score(y.values.flatten(), y_pred_train_poly)  # Menggunakan y.values.flatten() untuk menghindari shape mismatch
# print(f"\nR² Train: {r2_train:.4f}")

# Evaluasi MSE untuk data pelatihan
mse_train = mse_score(y.values.flatten(), y_pred_train_poly)
print(f"\nMSE Train: {mse_train:.4f}")

# # Evaluasi R² untuk data uji
# r2_test = r2_score(y_test.values.flatten(), y_pred_test_poly)
# print(f"\nR² Test: {r2_test:.4f}")

# Evaluasi MSE untuk data uji
mse_test = mse_score(y_test.values.flatten(), y_pred_test_poly)
print(f"\nMSE Test: {mse_test:.4f}")

# Evaluasi Q² LOO (Leave-One-Out Cross-Validation) pada data pelatihan
q2 = q2_loo(best_estimator_poly, X_train_poly.values, y.values)
print(f"\nQ² LOO: {q2:.4f}")

print("================================================")

# Menampilkan hasil QSAR untuk data pelatihan dan pengujian
train_results = qsar_param(y.values.flatten(), y_pred_train_poly)
test_results = qsar_param(y_test.values.flatten(), y_pred_test_poly)

print("\nTraining QSAR Results:")
for metric, value in train_results.items():
    print(f"{metric:<20}: {value:.4f}")

print("\nTest QSAR Results:")
for metric, value in test_results.items():
    print(f"{metric:<20}: {value:.4f}")



MSE Train: 0.5069

MSE Test: 1.4938


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(



Q² LOO: 0.6019

Training QSAR Results:
r2                  : 0.7723
k                   : 0.9196
k_dash              : 1.0492
r2r0                : 0.0039
r2r0_dash           : 0.1266
r0r0_dash           : 0.0948
rm2                 : 0.7299
rm2_dash            : 0.5308
rm2_bar             : 0.6303
d_rm2               : 0.1991

Test QSAR Results:
r2                  : 0.4953
k                   : 1.1687
k_dash              : 0.7379
r2r0                : 0.3619
r2r0_dash           : 0.0278
r0r0_dash           : 0.1655
rm2                 : 0.2856
rm2_dash            : 0.4372
rm2_bar             : 0.3614
d_rm2               : 0.1516


In [ ]:
# def validate_lengths(y, y_pred, name="y and y_pred"):
#     if len(y) != len(y_pred):
#         raise ValueError(f"Length mismatch in {name}: {len(y)} vs {len(y_pred)}")
#     else:
#         print(f"Length check passed for {name}: {len(y)}")


In [ ]:
# # Validasi panjang data sebelum evaluasi
# validate_lengths(y.values.flatten(), y_pred_train_poly, name="y_train and y_pred_train_poly")
# validate_lengths(y_test.values.flatten(), y_pred_test_poly, name="y_test and y_pred_test_poly")
